In [2]:
from mysql.connector import MySQLConnection, Error

In [5]:
def make_connection():
    """
    Make a connection to the school database.
    """
    try:
        conn = MySQLConnection(host = 'IES-ADS-ClassDB.sjsu.edu',
                               database = 'querycrew_wh',
                               user = 'querycrew_user',
                               password = 'Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [7]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [9]:
import sys
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QWidget, QVBoxLayout, QPushButton,
    QLabel, QComboBox, QMessageBox, QTableView, QHeaderView
)
from PyQt5.QtSql import QSqlDatabase, QSqlTableModel, QSqlQuery, QSqlQueryModel

class OperationalApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Query Crew Main Menu")
        self.resize(800, 600)

        # Database connection
        self.db = self.init_db()
        # Show the top-level menu
        self.init_menu()

    def init_db(self):
        db = QSqlDatabase.addDatabase("QMYSQL")
        db.setHostName("IES-ADS-ClassDB.sjsu.edu")
        db.setDatabaseName("querycrew_wh")
        db.setUserName("querycrew_user")
        db.setPassword("Pomegranate_746")
        if not db.open():
            QMessageBox.critical(None, "DB Connection Error", db.lastError().text())
            sys.exit(1)
        return db

    def init_menu(self):
        menu = QWidget()
        layout = QVBoxLayout(menu)

        greeting = QLabel("Hello, welcome to our app")
        greeting.setStyleSheet("font-size: 18px; font-weight: bold;")
        layout.addWidget(greeting)

        btn_customers = QPushButton("Click here for the customers portal")
        btn_customers.clicked.connect(self.show_customers_portal)
        layout.addWidget(btn_customers)

        btn_dealers = QPushButton("Click here for the dealers portal")
        btn_dealers.clicked.connect(self.show_dealers_portal)
        layout.addWidget(btn_dealers)

        self.setCentralWidget(menu)

    def show_dealers_portal(self):
        portal = QWidget()
        layout = QVBoxLayout(portal)

        title = QLabel("Dealers Portal")
        title.setStyleSheet("font-size: 16px; font-weight: bold;")
        layout.addWidget(title)

        btn_view_cust = QPushButton("View Customers")
        btn_view_cust.clicked.connect(lambda: self.show_table("Customers"))
        layout.addWidget(btn_view_cust)

        btn_monthly = QPushButton("Monthly EV Sales by Dealer")
        btn_monthly.clicked.connect(self.show_monthly_ev_sales)
        layout.addWidget(btn_monthly)

        btn_back = QPushButton("Back to Main Menu")
        btn_back.clicked.connect(self.init_menu)
        layout.addWidget(btn_back)

        self.setCentralWidget(portal)

    def show_customers_portal(self):
        portal = QWidget()
        layout = QVBoxLayout(portal)

        title = QLabel("Customers Portal")
        title.setStyleSheet("font-size: 16px; font-weight: bold;")
        layout.addWidget(title)

        btn_avg = QPushButton("Average Car Price by Make/Model")
        btn_avg.clicked.connect(self.show_avg_cost)
        layout.addWidget(btn_avg)

        btn_search = QPushButton("Search for a car")
        btn_search.clicked.connect(self.search_car)
        layout.addWidget(btn_search)

        btn_back = QPushButton("Back to Main Menu")
        btn_back.clicked.connect(self.init_menu)
        layout.addWidget(btn_back)

        self.setCentralWidget(portal)

    def show_avg_cost(self):
        sql = (
            "SELECT c.car_make AS Make, c.car_model AS Model, "
            "CONCAT('$', ROUND(AVG(c.price), 2)) AS Average_Car_Price "
            "FROM car_sales c "
            "WHERE c.car_make IS NOT NULL AND c.car_model IS NOT NULL AND c.price IS NOT NULL "
            "GROUP BY c.car_make, c.car_model "
            "ORDER BY c.car_make, c.car_model"
        )
        model = QSqlQueryModel(self)
        model.setQuery(sql, self.db)
        if model.lastError().isValid():
            QMessageBox.warning(self, "Query Error", model.lastError().text())
            return

        view_widget = QWidget()
        layout = QVBoxLayout(view_widget)
        view = QTableView()
        view.setModel(model)
        view.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        layout.addWidget(view)

        back_btn = QPushButton("Back to Customers Portal")
        back_btn.clicked.connect(self.show_customers_portal)
        layout.addWidget(back_btn)

        self.setCentralWidget(view_widget)

    def search_car(self):
        widget = QWidget()
        layout = QVBoxLayout(widget)
        layout.addWidget(QLabel("Select type of make:"))

        combo = QComboBox()
        q = QSqlQuery(self.db)
        q.exec_("SELECT DISTINCT ev_make FROM EV_arrivals ORDER BY ev_make")
        while q.next():
            combo.addItem(q.value(0))
        layout.addWidget(combo)

        back = QPushButton("Back to Customers Portal")
        back.clicked.connect(self.show_customers_portal)
        layout.addWidget(back)

        self.setCentralWidget(widget)

    def show_monthly_ev_sales(self):
        sql = (
            "SELECT e.dealer_id, dss.dealer_name, "
            "DATE_FORMAT(e.arrival_date, '%Y-%m') AS arrival_month, "
            "COUNT(*) AS ev_arrival_count "
            "FROM EV_arrivals e "
            "JOIN dealer_sales_summary dss ON e.dealer_id = dss.dealer_id "
            "GROUP BY e.dealer_id, dss.dealer_name, DATE_FORMAT(e.arrival_date, '%Y-%m') "
            "ORDER BY e.dealer_id, arrival_month"
        )
        model = QSqlQueryModel(self)
        model.setQuery(sql, self.db)
        err = model.lastError().text()
        if err:
            QMessageBox.warning(self, "Query Error", err)
            return

        widget = QWidget()
        layout = QVBoxLayout(widget)
        view = QTableView()
        view.setModel(model)
        view.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        layout.addWidget(view)

        back_btn = QPushButton("Back to Dealers Portal")
        back_btn.clicked.connect(self.show_dealers_portal)
        layout.addWidget(back_btn)

        self.setCentralWidget(widget)

    def show_table(self, tbl_name: str):
        model = QSqlTableModel(self, self.db)
        model.setTable(tbl_name)
        model.select()

        widget = QWidget()
        layout = QVBoxLayout(widget)
        view = QTableView()
        view.setModel(model)
        view.horizontalHeader().setSectionResizeMode(QHeaderView.Stretch)
        layout.addWidget(view)

        back_btn = QPushButton("Back to Dealers Portal")
        back_btn.clicked.connect(self.show_dealers_portal)
        layout.addWidget(back_btn)

        self.setCentralWidget(widget)

if __name__ == "__main__":
    app = QApplication(sys.argv)
    win = OperationalApp()
    win.show()
    sys.exit(app.exec_())


SystemExit: 0

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
cursor.close()
conn.close()